<img src="https://raw.githubusercontent.com/astrolabsoftware/fink-notebook-template/main/Fink_PrimaryLogo_WEB.jpg" width=400 />

# Fink case study: name resolver

## Goal

This notebook shows how to resolve object names using various databases.

## What is behind?

Giving names is a complex process, and often objects end up with multiple names or designations. At the era of big data, this is especially harder as the need to names million of objects quickly lead to non-intuitive designation process. We will not come up with a new naming scheme here, but rather offer a new service to explore existing names for a given object. Currently, you will be able to resolve ZTF object names in SIMBAD, the Transient Name Server, and Solar System databases known by the Quaero service from SSODNET -- and vice versa meaning given an object in one of those 3 databases, you will know if there is a corresponding ZTF objects. This service is expanding, and we will keep adding new source of information.

In [1]:
import io
import requests
import pandas as pd

## TNS to ZTF

I have a TNS identifier, are there ZTF objects corresponding?

In [18]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'tns',
    'name': 'SN 2023vwy'
  }
)

# Format output in a DataFrame
pdf = pd.read_json(io.BytesIO(r.content))
pdf

,d:declination,d:fullname,d:internalname,d:ra,d:redshift,d:type,key:time
0,52.219894,SN 2023vwy,ATLAS23url,47.765951,0.031,SN Ia,1704654335718
1,52.219894,SN 2023vwy,ZTF23abmwsrw,47.765951,0.031,SN Ia,1704654335718


You can also download the full table by specifying an empty name:

In [19]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'tns',
    'name': '',
    'nmax': 1000000
  }
)

pdf = pd.read_json(io.BytesIO(r.content))

## ZTF to TNS

I have a ZTF object name, are there counterparts in TNS?

In [20]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'tns',
    'reverse': True,
    'name': 'ZTF23abmwsrw'
  }
)

# Format output in a DataFrame
pdf = pd.read_json(io.BytesIO(r.content))
pdf

,d:declination,d:fullname,d:internalname,d:ra,d:redshift,d:type,key:time
0,52.219894,SN 2023vwy,ZTF23abmwsrw,47.765951,0.031,SN Ia,1704654335718


## SIMBAD to ZTF

I have an astronomical object name referenced in SIMBAD, are there counterparts in ZTF? As these objects can be extended, we typically provide coordinates, and then you need to run a conesearch:

In [21]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'simbad',
    'name': 'Markarian 2'
  }
)

if r.json() != []:
    print('Object found!')
    print(r.json())
    print()

    r = requests.post(
      'https://fink-portal.org/api/v1/explorer',
      json={
        'ra': r.json()[0]['jradeg'],
        'dec': r.json()[0]['jdedeg'],
        'radius': 60
      }
    )

    # Format output in a DataFrame
    pdf = pd.read_json(io.BytesIO(r.content))
    print('Object(s) in ZTF: {}'.format(pdf['i:objectId'].values))
else:
    print('No objects found')

Object found!
[{'name': 'Si=Simbad, all IDs (via url)', 'oid': 1579005, 'oname': 'Mrk 2', 'otype': 'GiG', 'jpos': '01:54:53.80 +36:55:04.6', 'jradeg': 28.7241958, 'jdedeg': 36.9179556, 'refPos': '2006AJ....131.1163S', 'z': None, 'MType': 'SBa', 'nrefs': 138}]

Object(s) in ZTF: ['ZTF18aabfjoi']


## ZTF to SIMBAD

I have a ZTF object name, are there counterparts in SIMBAD?

In [22]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'simbad',
    'reverse': True,
    'name': 'ZTF18aabfjoi'
  }
)
pdf = pd.read_json(io.BytesIO(r.content))
pdf

,d:cdsxmatch,i:candid,i:dec,i:jd,i:objectId,i:ra
0,GinGroup,1045306711115010048,36.917909,2.458800e+06,ZTF18aabfjoi,28.724092
1,GinGroup,1400219061115010048,36.917912,2.459155e+06,ZTF18aabfjoi,28.724130
2,GinGroup,1626461821115010048,36.917924,2.459381e+06,ZTF18aabfjoi,28.724110
3,GinGroup,1977476361115010048,36.917913,2.459732e+06,ZTF18aabfjoi,28.724100
4,GinGroup,2119325081115010048,36.917892,2.459874e+06,ZTF18aabfjoi,28.724094


## SSO to ZTF

I have a SSO name or number, are there ZTF objects corresponding? Note that by default the search has a `contain` strategy -- namely we will look for all objects whose name _contains_ the provided search term (this is not a exact name search). By default, we only returns the first 10 matches. We will add in the future an option to specify exact search as well.

In [23]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'ssodnet',
    'name': '624188'
  }
)

pdf = pd.read_json(io.BytesIO(r.content))
pdf

,i:name,i:number,i:ssnamenr
0,2002 MA6,624188,624188
1,2002 MA6,624188,2002MA6
2,2002 MA6,624188,2002MA06
3,2002 MA6,624188,624188


In [25]:
r = requests.post(
  'https://fink-portal.org/api/v1/sso',
  json={
    'n_or_d': ','.join(pdf['i:ssnamenr'].values),
    'columns': 'i:objectId,i:ra,i:dec,i:magpsf,i:sigmapsf,i:ssnamenr'
  }
)
sso = pd.read_json(io.BytesIO(r.content))
sso

,i:dec,i:magpsf,i:objectId,i:ra,i:sigmapsf,i:ssnamenr
0,40.087669,19.518000,ZTF23abidfxt,45.472807,0.134570,624188
1,40.253863,20.037786,ZTF23abilvaz,45.374426,0.172761,624188
2,40.263377,19.517801,ZTF23abiotst,45.365776,0.116226,624188
3,40.717564,19.452196,ZTF23abkempy,44.508594,0.087599,624188
4,40.718352,19.871597,ZTF23abkfquu,44.504111,0.144978,624188
5,40.760788,19.485823,ZTF23abkwjee,44.211183,0.093203,624188
6,40.777337,19.365993,ZTF23ablsjlg,43.858879,0.078582,624188
7,40.777324,19.830032,ZTF23ablthyz,43.850151,0.109873,624188
8,40.046433,19.359879,ZTF23abogmeh,40.809196,0.127990,624188
9,39.832539,19.788800,ZTF23aboqpks,40.349063,0.139298,624188


## ZTF to SSO

I have a ZTF object name, is there a counterpart in the SsODNet quaero database, and what are all the aliases?

In [28]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'ssodnet',
    'reverse': True,
    'name': 'ZTF23abpkzcn'
  }
)

if r.json() != []:
    name = r.json()[0]['i:ssnamenr']
    print('Asteroid counterpart found with designation {}'.format(name))
    print()

    r = requests.post(
      'https://fink-portal.org/api/v1/resolver',
      json={
        'resolver': 'ssodnet',
        'name': name
      }
    )
    
    pdf = pd.read_json(io.BytesIO(r.content))
    print('In the quaero database, this corresponds to: ')
    print(pdf)

Asteroid counterpart found with designation 8467

In the quaero database, this corresponds to: 
        i:name  i:number  i:ssnamenr
0  Benoitcarry      8467        8467
1  Benoitcarry      8467        8467
2    2002 VF83     84670       84670
3    2002 VF83     84670       84670
4    2002 VS83     84671       84671
5    2002 VS83     84671       84671
6    2002 VG84     84672       84672
7    2002 VG84     84672       84672
8    2002 VW86     84673       84673
9    2002 VW86     84673       84673


By default, the SSO resolver will perform an inexact search, meaning it will return the exact search plus closest matches. The number of closest matches is controlled by the parameter nmax, with default nmax=10. If you want to perform an exact search, just put nmax=1:

In [29]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'ssodnet',
    'reverse': True,
    'name': 'ZTF23abpkzcn'
  }
)

if r.json() != []:
    name = r.json()[0]['i:ssnamenr']
    print('Asteroid counterpart found with designation {}'.format(name))
    print()

    r = requests.post(
      'https://fink-portal.org/api/v1/resolver',
      json={
        'resolver': 'ssodnet',
        'name': name,
        'nmax': 1
      }
    )
    
    pdf = pd.read_json(io.BytesIO(r.content))
    print('In the quaero database, this corresponds to: ')
    print(pdf)

Asteroid counterpart found with designation 8467

In the quaero database, this corresponds to: 
        i:name  i:number  i:ssnamenr
0  Benoitcarry      8467        8467
1  Benoitcarry      8467        8467
